## Ground Control Stations

This notebook serves to get familiar with the simulator

In [1]:
from config import DATA_PATH, Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from planner import Plan
from planner.plans.auto import AutoPlan
from simulator import (
    QGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)
from simulator.gazebo.preview import GazMarker
from simulator.QGroundControl.qgc import QGCMarker
from simulator.sim import SimVehicle

clean()

## Simulation Positions
 

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

In [3]:
enu_homes = enu_origin.to_abs_all(base_homes)
gra_homes = gra_origin.to_abs_all(base_homes)
enu_wptrajs = [enu_home.to_abs_all(base_path) 
               for enu_home, base_path in zip(enu_homes, base_paths)]
gra_wptrajs = [gra_home.to_abs_all(base_path) 
               for gra_home, base_path in zip(gra_homes, base_paths)]

## Create Vehicles

In [4]:
sysids = [11,12,13,21,22]
colors=3*[Color.BLUE]+2*[Color.RED]

vehs:list[SimVehicle] = []
for sysid, enu_home, gra_wptraj, enu_wptraj, color in zip(
        sysids, enu_homes, gra_wptrajs, enu_wptrajs, colors):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )
    plan.save_basic_mission(
        sysid=sysid,
        gra_wps=GRAPose.unpose_all(gra_wptraj),
    )

    veh = SimVehicle(
        sysid=sysid,
        gcs_name=f'{color.name}_{color.emoji}',
        plan=plan,
        color=color,
        home=enu_home,
        waypoints=ENUPose.unpose_all(enu_wptraj),
    )
    vehs.append(veh)
    

## Visualizer

### Gazebo

In [5]:
gaz= Gazebo(gra_origin,world_path="simulator/gazebo/worlds/runway.world")
origin_gaz = GazMarker(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

### QGroundControl

In [6]:
qgc= QGC(gra_origin)
origin_qgc = QGCMarker(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)
qgc.markers.append(origin_qgc)

### No Visualizer

In [7]:
novis = NoVisualizer(gra_origin)

## Simulator

In [8]:
simulator = Simulator(
	visualizer=qgc,
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

simulator.show()

## Run

In [9]:
orac = simulator.launch()
orac.run()

23:40:13 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 305818)
23:40:13 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 305819)
23:40:13 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
23:40:13 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 2 GCSs
23:42:00 - Oracle ⚪ - INFO - UAV 12 completed mission and exited
23:42:01 - Oracle ⚪ - INFO - UAV 13 completed mission and exited
23:42:01 - Oracle ⚪ - INFO - Received DONE from GCS BLUE_🟦
23:42:01 - Oracle ⚪ - INFO - UAV 11 completed mission and exited
23:42:02 - Oracle ⚪ - INFO - UAV 22 completed mission and exited
23:42:03 - Oracle ⚪ - INFO - Received DONE from GCS RED_🟥
23:42:03 - Oracle ⚪ - INFO - UAV 21 completed mission and exited
23:42:03 - Oracle ⚪ - INFO - ✅ All GCS threads completed
23:42:03 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
